In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import csv
import codecs

In [ ]:
filename = "/Users/user/Desktop/outlier/Gwangjingu_parking_total.csv"

with codecs.open(filename, 'r', 'euc_kr', errors='ignore') as f:
    reader = csv.reader(f)

Apriori 알고리즘 -> 실패 ^^(원인 : 지표로서 사용하기 좋지 않음)

In [ ]:
# 데이터 불러오기
df = pd.read_csv(filename, encoding='cp949', usecols=['단속일시', '단속동', '단속장소', '위반내용'])

# 단속일시를 이용하여 요일 구하기
df['요일'] = pd.to_datetime(df['단속일시']).dt.dayofweek
# 시간대 구하기
df['시간대'] = pd.cut(pd.to_datetime(df['단속일시']).dt.hour, bins=12, labels=range(12))

df

,단속일시,단속동,단속장소,위반내용,요일,시간대
0,2015-01-01 06:44,화양동,119-4,주정차금지,3,3
1,2015-01-01 07:01,화양동,건대입구1번출구앞,32조내지 34조위반,3,3
2,2015-01-01 07:24,구의동,동서울터미널 주차장앞,32조내지 34조위반,3,3
3,2015-01-01 08:00,화양동,건대입구1번출구앞,32조내지 34조위반,3,4
4,2015-01-01 08:15,화양동,건대입구1번출구앞,32조내지 34조위반,3,4
...,...,...,...,...,...,...
490926,2023-02-28 20:43:00,군자동,군자빌딩,주정차금지(황색실선)구역,1,10
490927,2023-02-28 21:23:00,자양동,서울 광진구 자양동 659-32,보도,1,10
490928,2023-02-28 21:38:00,화양동,서울 광진구 능동로 154,보도,1,10
490929,2023-02-28 22:36:00,자양동,서울 광진구 자양동 12-10,교차로,1,11


In [ ]:
##### 데이터 전처리 #####
# 중복 데이터 제거
df.drop_duplicates(inplace=True)
print(df.shape)

# 단속장소 열을 문자열로 변환
df['단속장소'] = df['단속장소'].astype(str)

# 단속장소와 위반내용 컬럼에서 공백 제거
df['단속장소'] = df['단속장소'].str.replace(' ', '')
df['위반내용'] = df['위반내용'].str.replace(' ', '')

df

(453963, 6)


,단속일시,단속동,단속장소,위반내용,요일,시간대
0,2015-01-01 06:44,화양동,119-4,주정차금지,3,3
1,2015-01-01 07:01,화양동,건대입구1번출구앞,32조내지34조위반,3,3
2,2015-01-01 07:24,구의동,동서울터미널주차장앞,32조내지34조위반,3,3
3,2015-01-01 08:00,화양동,건대입구1번출구앞,32조내지34조위반,3,4
4,2015-01-01 08:15,화양동,건대입구1번출구앞,32조내지34조위반,3,4
...,...,...,...,...,...,...
490926,2023-02-28 20:43:00,군자동,군자빌딩,주정차금지(황색실선)구역,1,10
490927,2023-02-28 21:23:00,자양동,서울광진구자양동659-32,보도,1,10
490928,2023-02-28 21:38:00,화양동,서울광진구능동로154,보도,1,10
490929,2023-02-28 22:36:00,자양동,서울광진구자양동12-10,교차로,1,11


In [ ]:
# 요일과 시간대를 카테고리화
df['요일'] = df['요일'].astype('category')
df['시간대'] = df['시간대'].astype('category')

In [ ]:
# 시간대별 요일별 빈도수 계산
df['시간대'] = df['시간대'].astype(int)
df['요일'] = df['요일'].astype(int)
freq_table = pd.crosstab(df['시간대'], df['요일'], normalize = True)
freq_table

요일,0,1,2,3,4,5,6
시간대,,,,,,,
0,0.004060,0.003110,0.004047,0.004119,0.003978,0.007309,0.007549
1,0.000771,0.000826,0.001038,0.001115,0.001185,0.002000,0.002518
2,0.000359,0.000289,0.000408,0.000315,0.000482,0.000485,0.000498
3,0.008838,0.008289,0.008265,0.008598,0.008474,0.005395,0.005113
4,0.017378,0.016369,0.016197,0.016794,0.017634,0.004408,0.004791
5,0.016849,0.016056,0.015770,0.016673,0.016385,0.013768,0.013717
6,0.014466,0.015404,0.015387,0.015144,0.015682,0.009177,0.010377
7,0.024782,0.023819,0.021568,0.023991,0.022251,0.017211,0.022167
8,0.020872,0.020021,0.019977,0.020383,0.020955,0.013928,0.015911


In [ ]:
# Apriori 분석 수행
freq_table.columns = range(len(freq_table.columns))
frequent_itemsets = apriori(freq_table, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.1)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 0.004059802230578263

In [ ]:
print(freq_table)
print(frequent_itemsets)
print(rules)

In [ ]:
# Apriori 알고리즘 적용 :  위반내용
def apriori_generator(df):
    for time_name, time_data in df.groupby('시간대'):
        for day_name, day_data in time_data.groupby('요일'):
            transactions = day_data['위반내용'].apply(lambda x: x.split(',')).tolist()
            te = TransactionEncoder()
            te_ary = te.fit_transform(transactions)
            yield apriori(pd.DataFrame(te_ary, columns=te.columns_), min_support=0.001, use_colnames=True)

In [ ]:
# 연관 규칙 탐색
for frequent_itemsets in apriori_generator(df):
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    # rules를 활용한 분석 결과를 저장하거나 출력하는 등의 작업을 수행합니다.
    top_items = frequent_itemsets.nlargest(10, 'support')
    print(top_items)

# inf : 전제조건이 성립할 때 결과조건이 성립하지 않을 확률의 증가비율이 무한대라는 것 -> 다른 지표 사용

     support         itemsets
3   0.255562     (도로교통법 제32조)
19  0.158980   (주차금지(황색점선)구역)
15  0.141074          (주정차금지)
20  0.129137         (주차방법위반)
1   0.071622         (교통소통장애)
7   0.037439             (보도)
8   0.036354           (보도주차)
2   0.033641         (도로 모퉁이)
16  0.033641  (주정차금지(황색실선)구역)
21  0.023874           (횡단보도)
     support        itemsets
17  0.184136  (주차금지(황색점선)구역)
3   0.175637    (도로교통법 제32조)
13  0.136686         (주정차금지)
18  0.129603        (주차방법위반)
1   0.073654        (교통소통장애)
7   0.072238            (보도)
19  0.065864          (횡단보도)
2   0.043201        (도로 모퉁이)
8   0.034703          (보도주차)
5   0.019830         (버스정류소)
     support         itemsets
3   0.182907     (도로교통법 제32조)
14  0.178008          (주정차금지)
18  0.146434   (주차금지(황색점선)구역)
19  0.132281         (주차방법위반)
1   0.071312         (교통소통장애)
7   0.064780             (보도)
22  0.046271           (횡단보도)
2   0.039194         (도로 모퉁이)
8   0.038106           (보도주차)
15  0.023952  (주정차금지(황색실선)구역)
     support         

In [ ]:
data = pd.DataFrame(top_items)
data

# 지지도가 높은 상위 10개의 데이터 출력 시 상(32조 내지 34조 : 보행자,차량 우선순위/제한속도/신호준수)

,support,itemsets
18,0.215156,(주차금지(황색점선)구역)
0,0.159844,(32조내지 34조위반)
16,0.110136,(주정차금지(황색실선)구역)
15,0.099415,(주정차금지)
19,0.084308,(주차방법위반)
4,0.064815,(도로교통법 제32조)
20,0.055799,(횡단보도)
2,0.054581,(교통소통장애)
9,0.051170,(보도)
3,0.026803,(도로 모퉁이)


In [ ]:
association_rules(data, metric = 'confidence', min_threshold=0.001)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric


In [ ]:
# 연관 규칙 탐색
for frequent_itemsets in apriori_generator(df):
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    print(rules)
# convince -> inf : 전제조건이 성립할 때 결과조건이 성립하지 않을 확률의 증가비율이 무한대 -> 지표로 사용하기 적절하지 x

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
  antecedents consequents  antecedent support  consequent support   support  \
0      (황색점선)      (황색실선)            0.001089            0.001089  0.001089   
1      (황색실선)      (황색점선)            0.001089            0.001089  0.001089   

   confidence   lift  leverage  conviction  zhangs_metric  
0         1.0  918.5  0.001088         inf            1.0  
1         1.0  918.5  0.001088         inf            1.0  
Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
  antecedents consequents  antecedent support  consequent support   support  \
0    

In [ ]:
te = TransactionEncoder()
te_result = te.fit(df).transform(df)
df = pd.DataFrame(te_result)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,False,False,True,False,False,False,False,True,True,False,False,False,True,False
1,False,False,True,False,True,False,False,True,False,False,False,False,False,False
2,False,False,True,False,False,False,True,True,False,False,False,False,False,True
3,False,True,False,False,False,True,False,False,False,False,True,True,False,False
4,False,False,False,False,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453958,False,False,False,False,False,False,False,False,False,False,False,False,False,False
453959,False,False,False,False,False,False,False,False,False,False,False,False,False,False
453960,False,False,False,False,False,False,False,False,False,False,False,False,False,False
453961,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
itemset = apriori(df, use_colnames = True)
itemset

,support,itemsets


단속동으로 GROUPBY

In [ ]:
data = pd.DataFrame(df)
data.groupby('단속동').count()

,단속일시,단속원금,단속장소,위반내용,견인지시,데이터기준일자
단속동,,,,,,
강변북로,29,29,29,29,29,28
강변역로,67,67,67,67,67,67
강변역로2길,4,4,4,4,4,4
강변역로4길,230,230,230,230,230,230
광나루로,2040,2040,2040,2040,2040,2040
...,...,...,...,...,...,...
천호대로138길,209,209,209,209,209,209
천호대로140길,5,5,5,5,5,5
천호대로141길,32,32,32,32,32,32


In [ ]:
# 단속일시 년도별로 추출
df['단속일시'] = pd.to_datetime(df['단속일시'], format='%Y-%m-%d %H:%M:%S')

# '단속일시' 열에서 '2020'년도 데이터만 추출
df_2020 = df[df['단속일시'].dt.year == 2020]

print(df_2020.head())
print(df_2020.shape)

print('----------------------------------------------------------------------------------------------------------------')

# '단속일시' 열에서 '2021'년도 데이터만 추출
df_2021 = df[df['단속일시'].dt.year == 2021]

print(df_2021.head())
print(df_2021.shape)

print('----------------------------------------------------------------------------------------------------------------')

# '단속일시' 열에서 '2022'년도 데이터만 추출
df_2022 = df[df['단속일시'].dt.year == 2022]

print(df_2022.head())
print(df_2022.shape)

print('----------------------------------------------------------------------------------------------------------------')

# '단속일시' 열에서 '2023'년도 데이터만 추출
df_2023 = df[df['단속일시'].dt.year == 2023]

print(df_2023.head())
print(df_2023.shape)

----------------------------------------------------------------------------------------------------------------
                      단속일시   단속원금       단속동      단속장소    위반내용 견인지시     데이터기준일자
331897 2020-01-01 00:05:00  40000       군자로       152  주차방법위반  미견인  2022-06-10
331898 2020-01-01 00:18:00  40000      아차산로       334      보도  미견인  2022-06-10
331899 2020-01-01 00:23:00  40000   뚝섬로58가길  20 주차장막음  주차방법위반  미견인  2022-06-10
331900 2020-01-01 00:33:00  40000  자양번영로3나길   35 주차장앞  주차방법위반  미견인  2022-06-10
331901 2020-01-01 00:45:00  40000    동일로12길        28  주차방법위반  미견인  2022-06-10
(60545, 7)
----------------------------------------------------------------------------------------------------------------
                      단속일시   단속원금  단속동                    단속장소    위반내용 견인지시  \
392442 2021-01-01 01:28:00  40000  중곡동  서울 광진구 중곡동 296-29 주차장앞  교통소통장애  미견인   
392443 2021-01-01 02:21:00  40000  구의동         서울 광진구 구의로 22-1      보도  미견인   
392444 2021-01-01 02:26:00  40000  중곡동        서울 광진구